In [1]:
import matplotlib.pyplot as plt
import networkx as nx

G = nx.random_geometric_graph(200, 0.125)
# position is stored as node attribute data for random_geometric_graph
pos = nx.get_node_attributes(G, 'pos')

for n in pos:
    print(pos[n], "myname is jeff")

# find node near center (0.5,0.5)
dmin = 1
ncenter = 0
for n in pos:
    x, y = pos[n]
    d = (x - 0.5)**2 + (y - 0.5)**2
    if d < dmin:
        ncenter = n
        dmin = d

# color by path length from node near center
p = dict(nx.single_source_shortest_path_length(G, ncenter))

plt.figure(figsize=(8, 8))
nx.draw_networkx_edges(G, pos, nodelist=[ncenter], alpha=0.4)
nx.draw_networkx_nodes(G, pos, nodelist=list(p.keys()),
                       node_size=80,
                       node_color=list(p.values()),
                       cmap=plt.cm.Reds_r)

plt.xlim(-0.05, 1.05)
plt.ylim(-0.05, 1.05)
plt.axis('off')
plt.show()

[0.8128477847757912, 0.9484504235935727] myname is jeff
[0.1048204231446288, 0.8640723523097462] myname is jeff
[0.28637211965581033, 0.019964484315573405] myname is jeff
[0.5901278903917154, 0.6849033183905628] myname is jeff
[0.9238220003981962, 0.9103811853917988] myname is jeff
[0.6084905144944629, 0.9076302235287081] myname is jeff
[0.6537349966949727, 0.4067287867025954] myname is jeff
[0.18903001011267684, 0.46881619174092404] myname is jeff
[0.7307103535471317, 0.10140393199649156] myname is jeff
[0.6378118846765186, 0.5997475819671737] myname is jeff
[0.39162404444581733, 0.4149555530695639] myname is jeff
[0.12557161233707226, 0.03617901904516607] myname is jeff
[0.1303384339393342, 0.9688767108133792] myname is jeff
[0.4154342625454195, 0.07219885253307246] myname is jeff
[0.456201971236771, 0.2311777891953707] myname is jeff
[0.8252424298513504, 0.21399647957656764] myname is jeff
[0.6257982504320034, 0.9771428528492342] myname is jeff
[0.36432014815281044, 0.16290460205306

/usr/local/lib/python3.7/site-packages/networkx/drawing/nx_pylab.py:579: MatplotlibDeprecationWarning: 
The iterable function was deprecated in Matplotlib 3.1 and will be removed in 3.3. Use np.iterable instead.
  if not cb.iterable(width):


<Figure size 800x800 with 1 Axes>